# TESTING

In [93]:
import pandas as pd
print('pandas', pd.__version__)

import pickle
print('pickle', pickle.format_version)

from sklearn.model_selection import train_test_split, KFold, cross_validate, TimeSeriesSplit
from sklearn.metrics import mean_squared_error


from statsmodels.tsa.statespace.sarimax import SARIMAX

import numpy as np
import math

pandas 1.3.5
pickle 4.0


# Livello

In [94]:
X = pd.read_pickle('###', compression='zip')
df_sarima_livello = X[X.columns.difference(['Portata_Uscita', 'Year'])]
df_sarima_livello

,Data,Livello_Acqua,Pioggia_Totale,Temperatura_Zona_5,cc_1,cc_2,ss_1,ss_2
4820,2017-03-14,29.75,-0.391247,-0.255842,0.313921,-0.802907,0.949449,0.596104
4821,2017-03-15,29.74,-0.391247,-0.400104,0.297543,-0.822937,0.954709,0.568133
4822,2017-03-16,29.74,-0.391247,-0.270269,0.281076,-0.841993,0.959686,0.539489
4823,2017-03-17,29.75,-0.391247,-0.558793,0.264526,-0.860052,0.964378,0.510207
4824,2017-03-18,29.76,-0.391247,-0.371252,0.247898,-0.877093,0.968786,0.480321
...,...,...,...,...,...,...,...,...
6013,2020-06-19,29.85,-0.391247,0.609729,-0.976509,0.907139,0.215477,-0.420831
6014,2020-06-20,29.84,-0.391247,0.782844,-0.980071,0.921078,0.198648,-0.389378
6015,2020-06-21,29.83,-0.391247,1.064155,-0.983343,0.933927,0.181760,-0.357464
6016,2020-06-22,29.82,-0.391247,1.222843,-0.986324,0.945670,0.164818,-0.325128


## Model loading

In [95]:
with open('###', 'rb') as file:
    model_livello = pickle.load(file)
model_livello

## Model Performance

In [96]:
lista_colonne = ['Pioggia_Totale', 'Temperatura_Zona_5', 'cc_1','ss_1','cc_2','ss_2']
exog = df_sarima_livello[lista_colonne]

In [97]:
# Numero di previsioni nel test
# Massimo 150 perchè poi nel ciclo si fanno passi di 7 giorno e al massimo ci sono 1198 dati disponibili
n_test = 150
# Array per memorizzare errori
err = []

In [98]:
%%time
for i in range(0, n_test):
    # Definiamo le nuove variabili esterne
    # Future osservazioni distanti 7 passi 
    start = 4820 + i * 7
    
    # exog_2 e' la storia passata che serve
    exog_2 = exog.loc[start:(start+6)]
    exog_2.reset_index(inplace = True)
    exog_2 = exog_2[lista_colonne]
    
    # exog_3 sono le variabili esterne che servono per la previsione
    exog_3 = exog.loc[(start+7):(start+13)]
    exog_3.reset_index(inplace = True)
    exog_3 = exog_3[lista_colonne]

    new_obs = np.log(df_sarima_livello.loc[start:(start+6)]['Livello_Acqua'])
    new_obs = new_obs.reset_index()
    new_obs = new_obs['Livello_Acqua']
    
    
    # Applichiamo il modello salvato alla nuova serie
    fit_newobs = model_livello.apply(new_obs, exog = exog_2)
    
    predictions = np.exp(fit_newobs.forecast(steps = 7, exog = exog_3))
    error = (predictions.loc[13]-df_sarima_livello.loc[(start+6),'Livello_Acqua'])**2
    
    err.append(error)

Wall time: 1.9 s


In [99]:
# Media degli errori
np.mean(err)

0.05541559295397633

# PORTATA

In [100]:
X = pd.read_pickle('###', compression='zip')
df_sarima_portata = X[X.columns.difference(['Year'])]
df_sarima_portata

,Data,Livello_Acqua,Livello_Acqua_square,Pioggia_Totale,Portata_Uscita,Temperatura_Zona_5,Time,cc_1,cc_2,ss_1,ss_2
4820,2017-03-14,0.850746,0.723768,-0.391247,0.6,-0.255842,0.638961,0.313921,-0.802907,0.949449,0.596104
4821,2017-03-15,0.811260,0.658143,-0.391247,0.6,-0.400104,0.890577,0.297543,-0.822937,0.954709,0.568133
4822,2017-03-16,0.767387,0.588883,-0.391247,0.6,-0.270269,0.983028,0.281076,-0.841993,0.959686,0.539489
4823,2017-03-17,0.723514,0.523472,-0.391247,0.6,-0.558793,0.999254,0.264526,-0.860052,0.964378,0.510207
4824,2017-03-18,0.684028,0.467894,-0.391247,0.6,-0.371252,0.975687,0.247898,-0.877093,0.968786,0.480321
...,...,...,...,...,...,...,...,...,...,...,...
6013,2020-06-19,0.618218,0.382194,-0.391247,0.6,0.609729,0.719246,-0.976509,0.907139,0.215477,-0.420831
6014,2020-06-20,0.609444,0.371421,-0.391247,0.6,0.782844,0.719827,-0.980071,0.921078,0.198648,-0.389378
6015,2020-06-21,0.600669,0.360803,-0.391247,0.6,1.064155,0.720407,-0.983343,0.933927,0.181760,-0.357464
6016,2020-06-22,0.591894,0.350339,-0.391247,0.6,1.222843,0.720986,-0.986324,0.945670,0.164818,-0.325128


## Model Loading

In [101]:
with open('###', 'rb') as file:
    model_portata = pickle.load(file)
model_portata

## Model Performance

In [102]:
colonne_portata = ['Pioggia_Totale', 'Temperatura_Zona_5', 'cc_1','ss_1','cc_2','ss_2']
    

In [103]:
# Scegliamo le colonne definite sopra
exog = df_sarima_portata[colonne_portata]
# Nuovo array per memorizzare gli errori
err_portata = []

In [104]:
exog_2

,Pioggia_Totale,Temperatura_Zona_5,cc_1,ss_1,cc_2,ss_2
0,-0.391247,-1.200759,0.941397,0.337301,0.772456,0.635068
1,-0.391247,-0.905021,0.935455,0.353445,0.750154,0.661263
2,-0.189216,-1.222398,0.929237,0.369484,0.726964,0.686676
3,1.399732,-1.222398,0.922744,0.385413,0.702913,0.711276
4,-0.123692,-0.941087,0.915978,0.401229,0.678031,0.735034
5,-0.292962,-1.121414,0.908940,0.416926,0.652346,0.757922
6,0.400497,-0.746333,0.901634,0.432499,0.625889,0.779913


In [105]:
exog_3

,Pioggia_Totale,Temperatura_Zona_5,cc_1,ss_1,cc_2,ss_2
0,-0.391247,-0.681415,0.894061,0.447945,0.598691,0.800980
1,-0.325724,-0.804038,0.886224,0.463258,0.570784,0.821100
2,0.728115,-0.832890,0.878124,0.478434,0.542202,0.840248
3,1.033892,-0.587645,0.869764,0.493468,0.512978,0.858402
4,-0.391247,-0.486662,0.861147,0.508356,0.483147,0.875539
5,-0.325724,-0.501088,0.852275,0.523094,0.452745,0.891640
6,-0.385787,-0.652563,0.843151,0.537677,0.421806,0.906686


In [106]:
%%time
for i in range(0, n_test):
    # Definiamo le nuove variabili esterne
    # Future osservazioni distanti 7 passi 
    start = 4820 + i * 7
    # exog_2 e' la storia passata che serve
    exog_2 = exog.loc[start:(start+6)]
    exog_2.reset_index(inplace = True)
    exog_2 = exog_2[colonne_portata]
    
    #exog_3 sono le variabili esterne che servono per la previsione
    exog_3 = exog.loc[(start+7):(start+13)]
    exog_3.reset_index(inplace = True)
    exog_3 = exog_3[colonne_portata]
    
    new_obs = np.log(df_sarima_portata.loc[start:(start+6),'Portata_Uscita'])
    new_obs = new_obs.reset_index()
    new_obs = new_obs['Portata_Uscita']
 
    # Applichiamo il modello salvato alla nuova serie
    fit_newobs = model_livello.apply(new_obs, exog = exog_2)
   
    
    predictions = np.exp(fit_newobs.forecast(steps = 7, exog = exog_3))
    error = (predictions.loc[13]-df_sarima_portata.loc[(start+6),'Portata_Uscita'])**2
    
    err_portata.append(error)



Wall time: 2.12 s


In [107]:
np.mean(err_portata)

28.350255843257315